In [14]:
from pathlib import Path
import shutil


def organize_dcm_to_bids(src_root: str | Path, bids_root: str | Path, dcm_pattern_list=['T1']) -> None:
    """
    Copy dcm scan folders into a BIDS-style layout.

    Creates:
        <bids_root>/<patient>/ses-01/DCM/<AX_T1_scan_dir>/*
        <bids_root>/<patient>/ses-01/ANAT/

    Args
    ----
    src_root : str | Path
        Directory whose immediate subdirectories are patient folders.
    bids_root : str | Path
        Destination directory; “BIDS” level will be created if absent.
    dcm_pattern_list : list of str
        List of patterns to match scan directories. Default is ['T1'].

    Notes
    -----
    • Case-insensitive match for “AX_T1” in the scan directory name.  
    • Copies entire scan folder with `shutil.copytree(..., dirs_exist_ok=True)`.
    """
    src_root = Path(src_root).expanduser().resolve()
    bids_root = Path(bids_root).expanduser().resolve()

    for patient_dir in filter(Path.is_dir, src_root.iterdir()):
        dcm_dst = bids_root / 'bids' / ('sub-' + patient_dir.name) / "ses-01" / "DCM"
        anat_dst = bids_root / 'bids' / ('sub-' + patient_dir.name) / "ses-01" / "ANAT"
        dcm_dst.mkdir(parents=True, exist_ok=True)
        anat_dst.mkdir(parents=True, exist_ok=True)

        for scan_dir in filter(Path.is_dir, patient_dir.iterdir()):
            if any(x in scan_dir.name.upper() for x in dcm_pattern_list):
                shutil.copytree(scan_dir, dcm_dst / scan_dir.name, dirs_exist_ok=True)


In [15]:
organize_dcm_to_bids('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/DICOM_Files', '/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS', ['T1', 'MPR'])

In [16]:
from pathlib import Path
import subprocess


def run_dcm2nii_over_bids(
    bids_root: str | Path,
    dcm2nii_cmd: str = "dcm2niix",
) -> None:
    """
    Convert every /DCM folder in a BIDS tree to NIfTI and drop
    the output in the matching /ANAT folder.

    Layout expected
    ---------------
    <bids_root>/
        sub-XXX/
            ses-YYY/
                DCM/      ← raw DICOM files (already organised)
                ANAT/     ← will receive the NIfTI output

    Parameters
    ----------
    bids_root : str | Path
        Path to the BIDS directory.
    dcm2nii_cmd : str
        Executable for DICOM→NIfTI conversion (dcm2niix/dcm2nii).
    """
    bids_root = Path(bids_root).expanduser().resolve()

    # locate every DCM directory: sub-*/ses-*/DCM
    for dcm_dir in bids_root.glob("sub-*/ses-*/DCM"):
        if not dcm_dir.is_dir():
            continue

        anat_dir = dcm_dir.parent / "ANAT"
        anat_dir.mkdir(exist_ok=True)

        # run the converter; raise if it fails
        try:
            subprocess.run(
                [dcm2nii_cmd, "-b", "y", "-z", "y", "-o", anat_dir, str(dcm_dir)],
                check=True,
            )
        except subprocess.CalledProcessError as e:
            print(f"Error processing {dcm_dir}: {e}")
            continue

        # OPTIONAL — rename first *.nii* to sub-XXX_T1.nii.gz
        # nifti_files = sorted(anat_dir.glob("*.nii*"))
        # if len(nifti_files) == 1:
        #     subj_id = dcm_dir.parents[1].name.lstrip("sub-")
        #     nifti_files[0].rename(anat_dir / f"{subj_id}_T1.nii.gz")


In [18]:
run_dcm2nii_over_bids('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids', 'dcm2niix')

Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 1328 DICOM file(s)
Convert 28 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E32031778/ses-01/ANAT/DCM_Post_Ax_T1_Flash_20221215113712_22 (320x320x28x1)
Convert 180 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E32031778/ses-01/ANAT/DCM_Post_Sag_T2_3D_Space_Wave_20221215113712_27 (220x220x180x1)
Convert 52 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E32031778/ses-01/ANAT/DCM_Post_Sag_T1_3D_MPRAGE_Wave_20221215113712_24 (220x220x52x1)
Convert 180 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E32031778/ses-01/ANAT/DCM_Sag_T1_3D_MPRAGE_Wave_20221215113712_6 (220x220x180x1)
Slices not stacked: orientation varies (vNav or localizer?) [0.995833 -0.0553914 -0.0724501 0.059594 0.996583 0.0571907] != [0.999359 -0.0222655 0.028032 0.0281495 0.972527 -0.231082]
Convert 1 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy

Error: Unable to find any DICOM images in /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E17519160/ses-01/DCM (or subfolders 5 deep)
Error: Unable to find any DICOM images in /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E30757577/ses-01/DCM (or subfolders 5 deep)


Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Error processing /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E17519160/ses-01/DCM: Command '['dcm2niix', '-b', 'y', '-z', 'y', '-o', PosixPath('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E17519160/ses-01/ANAT'), '/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E17519160/ses-01/DCM']' returned non-zero exit status 2.
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Error processing /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E30757577/ses-01/DCM: Command '['dcm2niix', '-b', 'y', '-z', 'y', '-o', PosixPath('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E30757577/ses-01/ANAT'), '/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E30757577/ses-01/DCM']' returned non-zero exit status 2.


Error: Unable to find any DICOM images in /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E23892595/ses-01/DCM (or subfolders 5 deep)


Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Error processing /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E23892595/ses-01/DCM: Command '['dcm2niix', '-b', 'y', '-z', 'y', '-o', PosixPath('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E23892595/ses-01/ANAT'), '/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E23892595/ses-01/DCM']' returned non-zero exit status 2.
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Error processing /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E21444040/ses-01/DCM: Command '['dcm2niix', '-b', 'y', '-z', 'y', '-o', PosixPath('/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E21444040/ses-01/ANAT'), '/Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E21444040/ses-01/DCM']' returned non-zero exit status 2.


...

Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 656 DICOM file(s)
Convert 262 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36930369/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_8 (215x215x262x1)
Convert 38 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36930369/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_9 (215x215x38x1)
Convert 176 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36930369/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_6 (216x256x176x1)
Convert 180 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36930369/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_7 (215x215x180x1)
Conversion required 9.565084 seconds (7.539498 for core code).
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 3584 DICOM file(s)
Convert 332 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E19511015/ses-01/ANA

Error: Converted 3200 of 3584 files
Error: Unable to find any DICOM images in /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E42032160/ses-01/DCM (or subfolders 5 deep)


Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 234 DICOM file(s)
Convert 15 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E1262627/ses-01/ANAT/DCM_SAG_T1_THIN_POST_20160407091960_10 (256x256x15x1)
Convert 15 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E1262627/ses-01/ANAT/DCM_COR_T1_THIN_POST_20160407091960_9 (256x256x15x1)
Convert 13 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E1262627/ses-01/ANAT/DCM_SAG_T1_20160407091960_2 (320x320x13x1)
Convert 176 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E1262627/ses-01/ANAT/DCM_AX_3D_MPRAGE_T1_POST_20160407091960_11 (216x256x176x1)
Convert 15 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E1262627/ses-01/ANAT/DCM_COR_T1_THIN_20160407091960_7 (256x256x15x1)
Conversion required 4.210207 seconds (3.274465 for core code).
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-

Error: Converted 1512 of 2261 files


Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 708 DICOM file(s)
Convert 292 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36309605/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_2 (512x512x292x1)
Slices not stacked: orientation varies (vNav or localizer?) [0.994276 -0.0468028 -0.0960461 0.030168 0.98535 -0.167855] != [0.0470202 0.998894 0 -0.0959399 0.00451612 -0.995377]
Convert 179 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36309605/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_250_i00002 (568x568x179x1)
Convert 1 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36309605/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_250_i00001 (568x568x1x1)
Convert 235 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana/BIDS/bids/sub-E36309605/ses-01/ANAT/DCM_Value_Anonymized_99990101000000_251_i00002 (568x568x235x1)
Convert 1 DICOM as /Volumes/Expansion/datasets/MSA_Atrophy_Kurana